In [1]:
!pip install -q -U transformers[torch]
!pip install -q -U datasets
!pip install -q -U evaluate
!pip install -q -U tokenizers
!pip install -q -U bitsandbytes
!pip install -q rouge_score
!pip install -q -U peft
!pip install -q -U accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 77.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 12.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
import re
import random
import numpy as np
from scipy.special import softmax
import pprint

import bitsandbytes as bnb

import torch
import transformers
import evaluate
from datasets import Dataset, load_dataset

# For from-scratch T5 model
from transformers import T5TokenizerFast, T5Config, T5ForConditionalGeneration

# For pre-trained T5 model
from transformers import T5Tokenizer, T5ForConditionalGeneration  # this won't import twice, just noting here what's for each model

# For all T5 models
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq, Trainer, TrainingArguments

# For BLEURT (to load a trained model for evaluation)
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM


# For style classifier model (also for evaluating the seq2seq model output)
from transformers import BertTokenizer, BertForSequenceClassification, BitsAndBytesConfig,pipeline
from transformers import TrainingArguments, Trainer

import pandas as pd
from tqdm import tqdm

In [3]:
# This cell will authenticate you and mount your Drive in the Colab.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Path to data save in Drive
train = 'FairytaleQA_train.csv'
valid = 'FairytaleQA_valid.csv'
test = 'FairytaleQA_test.csv'
# path = 'drive/MyDrive/266_Danielle_Dylan_final_project/data/'
path = '/content/drive/MyDrive/266/FinalProject/data/'

In [5]:
train_filepath = path+train
valid_filepath = path+valid
test_filepath = path+test

train = load_dataset("csv",data_files=train_filepath,streaming=True)['train']
val = load_dataset("csv",data_files=valid_filepath,streaming=True)['train']
test = load_dataset("csv",data_files=test_filepath,streaming=True)['train']

In [6]:
from datasets import load_dataset, DatasetDict
# # train_ds = Dataset.from_pandas(train, split="train")
# # test_ds = Dataset.from_pandas(test, split="test")
# # valid_ds = Dataset.from_pandas(valid, split="valid")

# Combine into a single DatasetDict
ds = DatasetDict({
    "train": train,
    "test": test,
    "validation": val,
})

In [7]:
train = ds['train'].shuffle()
val = ds['validation'].shuffle()
test = ds['test'].shuffle()

In [8]:
#make ds list then back to hf ds
train = Dataset.from_pandas(pd.DataFrame(train))
val = Dataset.from_pandas(pd.DataFrame(val))
test = Dataset.from_pandas(pd.DataFrame(test))

In [9]:
VOCAB_SIZE = 15000
MAX_SEQUENCE_LENGTH = 512


In [10]:
# We prefix our tasks with "answer the question"
prefix = "Please answer this question: "
context = " Context: "

# Define the preprocessing function

def preprocess_function(data):
   """Add prefix to the sentences, tokenize the text, and set the labels"""
   # The "inputs" are the tokenized answer:
   data['full_input'] = [context + story + prefix + question for question,story in zip(data["question"],data['story_section'])]

   input_encoded = tokenizer.batch_encode_plus(data['full_input'],
                      max_length=MAX_SEQUENCE_LENGTH,
                      truncation=True,
                      padding='max_length',
                            return_tensors='pt')

   # The "labels" are the tokenized outputs:
   answers_encoded = tokenizer.batch_encode_plus(data["answer1"],
                      max_length=MAX_SEQUENCE_LENGTH,
                      truncation=True,
                      padding='max_length',
                      return_tensors='pt')

   return {'input_ids': input_encoded['input_ids'],
           'labels': answers_encoded['input_ids']}

In [11]:
# Configure bitsandbytes for 4-bit quantization
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [12]:
model_id = "google/flan-t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_id)

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [13]:
# Map the preprocessing function across our dataset
train_ex = train.filter(lambda x: x['ex_or_im'] == 'explicit')
val_ex = val.filter(lambda x: x['ex_or_im'] == 'explicit')
test_ex = test.filter(lambda x: x['ex_or_im'] == 'explicit')

# train_tokenized_ex = train_ex.map(preprocess_function, batched=True)
# val_tokenized_ex = val_ex.map(preprocess_function, batched=True)
# test_tokenized_ex = test_ex.map(preprocess_function, batched=True)

Filter:   0%|          | 0/8548 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1025 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1007 [00:00<?, ? examples/s]

In [14]:
# Map the preprocessing function across our dataset
train_im = train.filter(lambda x: x['ex_or_im'] == 'implicit')
val_im = val.filter(lambda x: x['ex_or_im'] == 'implicit')
test_im = test.filter(lambda x: x['ex_or_im'] == 'implicit')

# train_tokenized_im = train_im.map(preprocess_function, batched=True)
# val_tokenized_im = val_im.map(preprocess_function, batched=True)
# test_tokenized_im = test_im.map(preprocess_function, batched=True)

Filter:   0%|          | 0/8548 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1025 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1007 [00:00<?, ? examples/s]

In [15]:
def generate_output(model, tokenizer, data, **kwargs):

    all_outputs = []
    prefix = "Please answer this question: "
    context = " Context: "
    input_sentences = [context + sample['story_section'] + prefix + sample['question'] for sample in data]

    for i in range(int(len(input_sentences)) + 1):
        if i >= len(input_sentences):
            break

        inputs_encoded = tokenizer(input_sentences[i],truncation=True, padding=True, return_tensors='pt')
        output_ids = model.cuda().generate(input_ids=inputs_encoded['input_ids'].cuda(), **kwargs)
        generated_sentences = tokenizer.batch_decode(output_ids,
                                                     skip_special_tokens=True,
                                                     clean_up_tokenization_spaces=False)
        all_outputs.extend(generated_sentences)

    return all_outputs

In [16]:
# metrics
rouge = evaluate.load('rouge')

bleu = evaluate.load("bleu")

bleurt_tokenizer = AutoTokenizer.from_pretrained("Elron/bleurt-base-512")
bleurt_model = AutoModelForSequenceClassification.from_pretrained("Elron/bleurt-base-512")

tokenizer_config.json:   0%|          | 0.00/321 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/779 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [17]:
%%time

# Store results for aggregate scoring

def calculate_eval_metrics(dataset, model, tokenizer, prefix="", **kwargs):

  # print("Starting Evaluation")
  # # Translate original texts
  # answers = generate_output(model, tokenizer, data, **kwargs)
  answers = generate_output(model, tokenizer, dataset, **kwargs)
  print("Answers generated")

  results = []
  bleurt_scores = []
  for idx, sample in enumerate(tqdm(dataset)):

      # # if idx >= 10:  # Stop after processing 10 samples
      # #   return pd.DataFrame(results)
      # #   break

      # prefix = "Please answer this question: "
      # context = " Context: "

      # question = prefix + sample['question'] + context + sample['story_section']

      # # Generate summary via the pipeline
      # outputs = pipe(
      #                     question,
      #                     max_new_tokens=500,
      # )


      answer = answers[idx]


      if answer.isspace():
        print("\nANSWER IS SPACE for question:",sample['question'],"\nReference:", sample['answer1'])
        print("\nanswer:",[answer])
        answer = 'blank'

      # print("\nbegin answer:", answer, ":end answer")
      # print("\n",[answer],"****")

      # Evaluation
      predictions = [answer]
      reference1 = [sample['answer1']]
      reference2 = [sample['answer2']]

      # Calculate ROUGE scores
      rouge_scores1 = rouge.compute(predictions=predictions, references=reference1)
      if reference1 != reference2:
        rouge_scores2 = rouge.compute(predictions=predictions, references=reference2)
      else:
        rouge_scores2 = rouge_scores1
      rouge_scores1 = {k+'_a1': v for k, v in rouge_scores1.items()}
      rouge_scores2 = {k+'_a2': v for k, v in rouge_scores2.items()}


      # Calculate BLEU scores
      bleu_scores1 = bleu.compute(predictions=predictions, references=reference1,smooth=True)
      if reference1 != reference2:
        bleu_scores2 = bleu.compute(predictions=predictions, references=reference2,smooth=True)
      else:
        bleu_scores2 = bleu_scores1
      bleu_scores1 = {k+'_a1': v for k, v in bleu_scores1.items()}
      bleu_scores2 = {k+'_a2': v for k, v in bleu_scores2.items()}

      # Calculate BLEURT scores
      with torch.no_grad():
        bleurt_score_a1 = bleurt_model(**bleurt_tokenizer(answer,
                                                sample['answer1'],
                                                truncation=True,
                                                max_length=512,
                                                padding='max_length',
                                                return_tensors='pt'))[0].squeeze().numpy()
        if reference1 != reference2:
          bleurt_score_a2 = bleurt_model(**bleurt_tokenizer(answer,
                                                sample['answer2'],
                                                truncation=True,
                                                max_length=512,
                                                padding='max_length',
                                                return_tensors='pt'))[0].squeeze().numpy()
        else:
          bleurt_score_a2 = bleurt_score_a1


      # Store results
      results.append({
          'id': idx,
          'question': sample['question'],
          'ex_or_im': sample['ex_or_im'],
          'story_section': sample['story_section'][:500],  # Store truncated text for readability
          'reference_answer1': sample['answer1'],
          'reference_answer2': sample['answer2'],
          'generated_answer': answer,
          **rouge_scores1,
          **rouge_scores2,
          **bleu_scores1,
          **bleu_scores2,
          'bleurt_score_a1': bleurt_score_a1,
          'bleurt_score_a2': bleurt_score_a2,
      })

      # Print progress update every 10 samples
      if (idx + 1) % 100 == 0:
          print(f"\nProcessed {idx + 1} samples; Results length: {len(results)}")
          print(f"Latest Answer1 ROUGE-1: {rouge_scores1['rouge1_a1']:.4f}")

  return pd.DataFrame(results)


CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 8.58 µs


In [18]:
# def calculate_eval_metrics(data, model, tokenizer, prefix="", **kwargs):
#     print("Starting Evaluation")
#     # Translate original texts
#     answers = generate_output(model, tokenizer, data, **kwargs)
#     print("Answers generated")

#     # Calculate ROUGE scores
#     rouge_results = rouge.compute(predictions=answers, references=data['answer1'])
#     print('ROUGE: ', rouge_results)

#     # Calculate BLEURT scores
#     bleurt_scores = []
#     for i in range(int(len(answers)) + 1):
#         if i >= len(answers):
#             break

#         with torch.no_grad():
#             scores = bleurt_model(**bleurt_tokenizer(data['answer1'][i],
#                                                      answers[i],
#                                                      truncation=True,
#                                                      max_length=MAX_SEQUENCE_LENGTH,
#                                                      padding='max_length',
#                                                      return_tensors='pt'))[0].squeeze().numpy()
#             if scores.shape:
#                 bleurt_scores.extend(scores)
#             else:  # Happens when there was only one example in the last batch
#                 bleurt_scores.append(float(scores))

#     print('BLEURT: ', np.mean(bleurt_scores))

#     return answers

## Experiments with Initial T5 trained on question type models
Run experiments for model trainings later

In [ ]:
# load implicit model
model_im_checkpoint_filepath = '/content/drive/MyDrive/266/FinalProject/checkpoints/t5_model_im'
model_im = AutoModelForSeq2SeqLM.from_pretrained(model_im_checkpoint_filepath)

# load explicit model
model_ex_checkpoint_filepath = '/content/drive/MyDrive/266/FinalProject/checkpoints/t5_model_ex'
model_ex = AutoModelForSeq2SeqLM.from_pretrained(model_ex_checkpoint_filepath)

In [ ]:
batch_size = 16
num_epochs = 3

In [ ]:
%%time

# Experiment 1 - Validation Explicit

generate_kwargs = {

    'num_beams': 4,
    'do_sample':True,
    'top_k':100,
    'top_p':1,
    'temperature':0.2,
    'no_repeat_ngram_size':2
}

val_answers_ex = calculate_eval_metrics(
    val_ex,
    model_ex,
    tokenizer,
    batch_size,
    **generate_kwargs
)
results_path = '/content/drive/MyDrive/266/FinalProject/results/'  #DANIELLE
val_answers_ex.to_csv(results_path+f'explicit_val_1.csv', index=False)

Answers generated


  0%|          | 0/744 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
 13%|█▎        | 100/744 [04:10<29:00,  2.70s/it]


Processed 100 samples; Results length: 100
Latest Answer1 ROUGE-1: 0.0000


 27%|██▋       | 200/744 [08:40<21:43,  2.40s/it]


Processed 200 samples; Results length: 200
Latest Answer1 ROUGE-1: 0.1333


...


Processed 600 samples; Results length: 600
Latest Answer1 ROUGE-1: 1.0000


 94%|█████████▍| 700/744 [31:17<02:14,  3.06s/it]


Processed 700 samples; Results length: 700
Latest Answer1 ROUGE-1: 0.8333


100%|██████████| 744/744 [33:16<00:00,  2.68s/it]

CPU times: user 36min 21s, sys: 9.39 s, total: 36min 30s
Wall time: 36min 57s


In [ ]:
%%time

# Experiment 2 - Validation Implicit

generate_kwargs = {

    'num_beams': 4,
    'do_sample':True,
    'top_k':100,
    'top_p':1,
    'temperature':0.2,
    'no_repeat_ngram_size':2
}

val_answers_im = calculate_eval_metrics(
    val_im,
    model_im,
    tokenizer,
    batch_size,
    **generate_kwargs
)

results_path = '/content/drive/MyDrive/266/FinalProject/results/'  #DANIELLE
val_answers_im.to_csv(results_path+f'implicit_val_2.csv', index=False)

Answers generated


  0%|          | 0/281 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
 36%|███▌      | 100/281 [03:36<07:07,  2.36s/it]


Processed 100 samples; Results length: 100
Latest Answer1 ROUGE-1: 0.1111


 71%|███████   | 200/281 [07:11<02:42,  2.01s/it]


Processed 200 samples; Results length: 200
Latest Answer1 ROUGE-1: 0.0000


100%|██████████| 281/281 [10:08<00:00,  2.17s/it]

CPU times: user 11min 25s, sys: 4.01 s, total: 11min 29s
Wall time: 11min 38s


In [ ]:
%%time

# Experiment 3 - Validation Implicit

generate_kwargs = {

    'num_beams': 1,
    'do_sample':False,
    'top_k':50,
    'top_p':1,
    'temperature':1,
    'no_repeat_ngram_size':0
}

val_answers_im = calculate_eval_metrics(
    val_im,
    model_im,
    tokenizer,
    batch_size,
    **generate_kwargs
)

results_path = '/content/drive/MyDrive/266/FinalProject/results/'  #DANIELLE
val_answers_im.to_csv(results_path+f'implicit_val_3.csv', index=False)

Answers generated


  0%|          | 0/281 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
 36%|███▌      | 100/281 [03:41<06:44,  2.24s/it]


Processed 100 samples; Results length: 100
Latest Answer1 ROUGE-1: 0.1429


 71%|███████   | 200/281 [07:19<02:47,  2.06s/it]


Processed 200 samples; Results length: 200
Latest Answer1 ROUGE-1: 0.0000


100%|██████████| 281/281 [10:15<00:00,  2.19s/it]

CPU times: user 11min, sys: 3.38 s, total: 11min 3s
Wall time: 11min 5s


In [ ]:
%%time

# Experiment 4 - Validation Explicit

generate_kwargs = {

    'num_beams': 1,
    'do_sample':False,
    'top_k':50,
    'top_p':1,
    'temperature':1,
    'no_repeat_ngram_size':0
}

val_answers_ex = calculate_eval_metrics(
    val_ex,
    model_ex,
    tokenizer,
    batch_size,
    **generate_kwargs
)
results_path = '/content/drive/MyDrive/266/FinalProject/results/'  #DANIELLE
val_answers_ex.to_csv(results_path+f'explicit_val_4.csv', index=False)

Answers generated


  0%|          | 0/744 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
 13%|█▎        | 100/744 [03:08<16:01,  1.49s/it]


Processed 100 samples; Results length: 100
Latest Answer1 ROUGE-1: 0.2222


 27%|██▋       | 200/744 [06:07<15:41,  1.73s/it]


Processed 200 samples; Results length: 200
Latest Answer1 ROUGE-1: 0.0000


...


Processed 600 samples; Results length: 600
Latest Answer1 ROUGE-1: 1.0000


 94%|█████████▍| 700/744 [21:30<01:14,  1.68s/it]


Processed 700 samples; Results length: 700
Latest Answer1 ROUGE-1: 0.0000


100%|██████████| 744/744 [22:53<00:00,  1.85s/it]


CPU times: user 24min 37s, sys: 6.51 s, total: 24min 44s
Wall time: 25min 3s


# Experiments on Various Implicit Trained Models Using TEST
Models trained and checkpointed in `266_Danielle_Dylan_T5_SpecializedFineTuning_Implicit_Only.ipynb`

*   t5_model_im_Experiments1 - a reruned version of the im_model used for experiments 2&3 above
*   t5_model_im_Experiments2 - Loss ~25
*   t5_model_im_Experiments3 - Loss ~4.6
*   t5_model_im_Experiments4 - Failed
*   t5_model_im_Experiments5 - Loss ~4.4




In [18]:
%%time

# Implicit Model 5 Experiment 1 - Test Data

# load implicit model
im_model_experiment = 't5_model_im_Experiments5'
model_im_checkpoint_filepath = f'/content/drive/MyDrive/266/FinalProject/checkpoints/{im_model_experiment}'
model_im = AutoModelForSeq2SeqLM.from_pretrained(model_im_checkpoint_filepath)


batch_size = 16
num_epochs = 10


generate_kwargs = {

    'num_beams': 4,
    'do_sample':True,
    'top_k':100,
    'top_p':1,
    'temperature':0.2,
    'no_repeat_ngram_size':2
}

test_answers_im = calculate_eval_metrics(
    test_im,
    model_im,
    tokenizer,
    batch_size,
    **generate_kwargs
)
results_path = '/content/drive/MyDrive/266/FinalProject/results/'
test_answers_im.to_csv(results_path+f't5_model_im_experiments5_test_1.csv', index=False)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Answers generated


  0%|          | 0/253 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
 40%|███▉      | 100/253 [03:35<05:50,  2.29s/it]


Processed 100 samples; Results length: 100
Latest Answer1 ROUGE-1: 0.0000


 79%|███████▉  | 200/253 [07:16<02:14,  2.54s/it]


Processed 200 samples; Results length: 200
Latest Answer1 ROUGE-1: 0.0000


100%|██████████| 253/253 [09:08<00:00,  2.17s/it]


CPU times: user 10min 24s, sys: 5.08 s, total: 10min 29s
Wall time: 10min 48s


In [19]:
%%time

# Implicit Model 3 Experiment 1 - Test Data

# load implicit model
im_model_experiment = 't5_model_im_Experiments3'
model_im_checkpoint_filepath = f'/content/drive/MyDrive/266/FinalProject/checkpoints/{im_model_experiment}'
model_im = AutoModelForSeq2SeqLM.from_pretrained(model_im_checkpoint_filepath)


batch_size = 16
num_epochs = 5


generate_kwargs = {

    'num_beams': 4,
    'do_sample':True,
    'top_k':100,
    'top_p':1,
    'temperature':0.2,
    'no_repeat_ngram_size':2
}

test_answers_im = calculate_eval_metrics(
    test_im,
    model_im,
    tokenizer,
    batch_size,
    **generate_kwargs
)
results_path = '/content/drive/MyDrive/266/FinalProject/results/'
test_answers_im.to_csv(results_path+f't5_model_im_experiments3_test_1.csv', index=False)

Answers generated


 40%|███▉      | 100/253 [03:37<05:56,  2.33s/it]


Processed 100 samples; Results length: 100
Latest Answer1 ROUGE-1: 0.2500


 79%|███████▉  | 200/253 [07:15<01:53,  2.14s/it]


Processed 200 samples; Results length: 200
Latest Answer1 ROUGE-1: 0.0000


100%|██████████| 253/253 [09:04<00:00,  2.15s/it]

CPU times: user 9min 53s, sys: 3.35 s, total: 9min 56s
Wall time: 10min 12s


In [19]:
%%time

# Implicit Model 2 Experiment 1 - Test Data

# load implicit model
im_model_experiment = 't5_model_im_Experiments2'
model_im_checkpoint_filepath = f'/content/drive/MyDrive/266/FinalProject/checkpoints/{im_model_experiment}'
model_im = AutoModelForSeq2SeqLM.from_pretrained(model_im_checkpoint_filepath)


batch_size = 16
num_epochs = 9


generate_kwargs = {

    'num_beams': 4,
    'do_sample':True,
    'top_k':100,
    'top_p':1,
    'temperature':0.2,
    'no_repeat_ngram_size':2
}

test_answers_im = calculate_eval_metrics(
    test_im,
    model_im,
    tokenizer,
    batch_size,
    **generate_kwargs
)
results_path = '/content/drive/MyDrive/266/FinalProject/results/'
test_answers_im.to_csv(results_path+f't5_model_im_experiments2_test_1.csv', index=False)

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Answers generated


...


Processed 100 samples; Results length: 100
Latest Answer1 ROUGE-1: 0.0000


 79%|███████▉  | 200/253 [12:56<03:28,  3.93s/it]


Processed 200 samples; Results length: 200
Latest Answer1 ROUGE-1: 0.0000


100%|██████████| 253/253 [16:16<00:00,  3.86s/it]


CPU times: user 17min 16s, sys: 7.37 s, total: 17min 23s
Wall time: 17min 49s


In [ ]:
def clear_gpu_memory():
    import gc
    import torch

    gc.collect()
    torch.cuda.empty_cache()
    print("Cleared GPU memory.")

# Usage
del model_im, tokenizer, trainer
clear_gpu_memory()
